# 🌌 x.titan: Autonomous Trading Brain - Kaggle Training

**Version**: 2026.1.3 (Fixed)
**Source**: GitHub → https://github.com/planetazul3/x.titan

## ⚙️ Requirements
1. **GPU Accelerator**: Enable in Kaggle Settings (T4 or P100)
2. **Kaggle Secrets**: Add `DERIV_API_TOKEN` for data download
3. **Internet**: Enable for GitHub clone + pip install

## 1. Clone Repository from GitHub

In [ ]:
import os
import sys
from pathlib import Path

REPO_URL = "https://github.com/planetazul3/x.titan.git"
WORKING_DIR = Path("/kaggle/working/x.titan")

if not WORKING_DIR.exists():
    print("📥 Cloning x.titan from GitHub...")
    !git clone --depth 1 {REPO_URL} {WORKING_DIR}
else:
    print("✅ Repository already exists")

os.chdir(WORKING_DIR)
sys.path.insert(0, str(WORKING_DIR))
print(f"📂 Working directory: {os.getcwd()}")

## 2. TA-Lib Installation (C Library)
**Note**: Using single-threaded make to avoid race conditions.

In [ ]:
%%bash
set -e

if [ ! -f /usr/include/ta-lib/ta_defs.h ]; then
    echo "📦 Installing TA-Lib..."
    apt-get update -qq && apt-get install -y -qq build-essential wget
    wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    tar -xzf ta-lib-0.4.0-src.tar.gz
    cd ta-lib
    ./configure --prefix=/usr > /dev/null
    make > /dev/null  # Single-threaded to avoid race condition
    make install > /dev/null
    cd ..
    rm -rf ta-lib ta-lib-0.4.0-src.tar.gz
    echo "✅ TA-Lib compiled successfully"
else
    echo "✅ TA-Lib already installed"
fi

# Verify installation
if [ -f /usr/lib/libta_lib.so ]; then
    echo "✅ TA-Lib library found"
else
    echo "❌ TA-Lib installation failed!"
    exit 1
fi

## 3. Python Dependencies

In [ ]:
print("📥 Installing Python packages...")
!pip install -q TA-Lib pandas numpy torch tqdm pydantic pydantic-settings python-dotenv pyarrow

# Install forked deriv-api in editable mode
import subprocess
result = subprocess.run(
    ["pip", "install", "-q", "-e", "./python-deriv-api"],
    capture_output=True, text=True
)
if result.returncode != 0:
    print("⚠️ Local install failed, trying from GitHub...")
    !pip install -q git+https://github.com/planetazul3/python-deriv-api.git

# Verify critical imports
try:
    import talib
    import deriv_api
    print("✅ Dependencies installed and verified")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    raise

## 4. Environment Configuration (API Key via Kaggle Secrets)

**⚠️ IMPORTANT**: Add your Deriv API token in Kaggle:
1. Click **Add-ons** → **Secrets**
2. Add key: `DERIV_API_TOKEN` with your token value

In [ ]:
# Load API token from Kaggle secrets
DERIV_API_TOKEN = ""
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    DERIV_API_TOKEN = secrets.get_secret("DERIV_API_TOKEN")
    if DERIV_API_TOKEN:
        print("✅ DERIV_API_TOKEN loaded from Kaggle secrets")
    else:
        print("⚠️ DERIV_API_TOKEN is empty - data download will fail")
except Exception as e:
    print(f"⚠️ Could not load secret: {e}")
    print("   Add DERIV_API_TOKEN via Kaggle Add-ons → Secrets")

# Create .env file with all configuration
env_content = f'''# x.titan Configuration (Generated by Kaggle Notebook)
ENVIRONMENT=test
DERIV_API_TOKEN={DERIV_API_TOKEN}
DERIV_APP_ID=1089

# Trading
TRADING__SYMBOL=R_100
TRADING__STAKE_AMOUNT=10.0

# Probability Thresholds
THRESHOLDS__CONFIDENCE_THRESHOLD_HIGH=0.80
THRESHOLDS__LEARNING_THRESHOLD_MAX=0.70
THRESHOLDS__LEARNING_THRESHOLD_MIN=0.50

# TFT Hyperparameters (GPU Optimized)
HYPERPARAMS__USE_TFT=True
HYPERPARAMS__LEARNING_RATE=0.0007
HYPERPARAMS__BATCH_SIZE=128
HYPERPARAMS__LSTM_HIDDEN_SIZE=256
HYPERPARAMS__CNN_FILTERS=128
HYPERPARAMS__LATENT_DIM=64
HYPERPARAMS__DROPOUT_RATE=0.2
HYPERPARAMS__EPOCHS=30

# Data Shapes
DATA_SHAPES__SEQUENCE_LENGTH_TICKS=1000
DATA_SHAPES__SEQUENCE_LENGTH_CANDLES=200
DATA_SHAPES__WARMUP_STEPS=50
'''

with open('.env', 'w') as f:
    f.write(env_content)
print("✅ .env file created")

## 5. GPU Detection & Optimization

In [ ]:
import torch

print("=" * 50)
print("GPU DETECTION")
print("=" * 50)

BATCH_SIZE = 16  # Default for CPU

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"   VRAM: {gpu_mem:.1f} GB")
    
    # Optimize for available VRAM
    if gpu_mem >= 15:
        BATCH_SIZE = 256
    elif gpu_mem >= 8:
        BATCH_SIZE = 128
    else:
        BATCH_SIZE = 64
    
    print(f"   Batch size: {BATCH_SIZE}")
    !nvidia-smi
else:
    print("⚠️ NO GPU DETECTED - Training will be very slow!")

os.environ['HYPERPARAMS__BATCH_SIZE'] = str(BATCH_SIZE)

## 6. Data Download (12 Months)
**Note**: This step requires a valid `DERIV_API_TOKEN` in Kaggle secrets.

In [ ]:
DATA_PATH = Path('./data_cache')

# Check if data already exists
tick_files = list(DATA_PATH.glob('**/ticks/*.parquet')) if DATA_PATH.exists() else []

if len(tick_files) >= 6:
    print(f"✅ Found {len(tick_files)} tick files - skipping download")
    DATA_READY = True
else:
    print("📡 Downloading 12 months of historical data...")
    print("   (This takes 10-15 minutes)")
    
    result = !python scripts/download_data.py --months 12 --symbol R_100 --output data_cache 2>&1
    for line in result:
        print(line)
    
    # Verify download succeeded
    tick_files = list(DATA_PATH.glob('**/ticks/*.parquet')) if DATA_PATH.exists() else []
    if len(tick_files) >= 6:
        print(f"\n✅ Download complete: {len(tick_files)} tick files")
        DATA_READY = True
    else:
        print("\n❌ DATA DOWNLOAD FAILED")
        print("   Check that DERIV_API_TOKEN is set correctly in Kaggle secrets")
        print("   Go to Add-ons → Secrets → Add 'DERIV_API_TOKEN'")
        DATA_READY = False

## 7. Training Pipeline
**Note**: This cell will only run if data download succeeded.

In [ ]:
import time

if not DATA_READY:
    print("❌ SKIPPING TRAINING - No data available")
    print("   Fix the data download step first")
else:
    # Ensure checkpoints directory exists
    Path('checkpoints').mkdir(exist_ok=True)
    
    print("🚀 STARTING TRAINING")
    print("=" * 50)
    start_time = time.time()
    
    # Execute training
    !python scripts/train.py \
        --data-path data_cache \
        --epochs 30 \
        --checkpoint-dir checkpoints
    
    elapsed = time.time() - start_time
    print(f"\n⏱️ Training completed in {elapsed/60:.1f} minutes")

## 8. Verify & Export Model

In [ ]:
import shutil
from datetime import datetime

best_model = Path('checkpoints/best_model.pt')

if not best_model.exists():
    print("❌ No checkpoint found!")
    print("   Training may not have completed successfully")
else:
    size_mb = best_model.stat().st_size / 1e6
    print(f"✅ best_model.pt: {size_mb:.1f} MB")
    
    # Validate checkpoint
    !python tools/verify_checkpoint.py --checkpoint checkpoints/best_model.pt
    
    # Create downloadable bundle
    ts = datetime.now().strftime('%Y%m%d_%H%M')
    bundle = f'xtitan_model_{ts}'
    shutil.make_archive(bundle, 'zip', root_dir='.', base_dir='checkpoints')
    print(f"\n📦 Download: {bundle}.zip from Output panel")